![title](../img/ESPN.png)

In [27]:
%run ./PProcess0.ipynb
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 111.0.5563
Get LATEST driver version for 111.0.5563
Driver [C:\Users\Valentin\.wdm\drivers\chromedriver\win32\111.0.5563.64\chromedriver.exe] found in cache


In [37]:
import datetime
URL_prefix = "https://www.espn.co.uk/football/table/_/league/"
Competitions = [
                "ger.1", "ger.2", #Germany
                "ita.1", "ita.2", #Italy
                "fra.1", "fra.2", #France
                "esp.1", "esp.2", #Spain
                "eng.1", "eng.2", #England
                "por.1", #"por.2", #Portugal
                "sco.1", #Scotland
                "den.1", #Denmark
                "aut.1", #Austria
                "sui.1", #Suisse
                "bel.1", #Belgique
                "ned.1", #Pays-Bas
                "nor.1", #Norvege
                "swe.1", #Suede
                "tur.1", #Turquie
                "gre.1", #Grèce
                "rou.1", #Roumanie
                "isr.1", #Israël
                "irl.1", #Irland
                "nir.1", #North Irland
                "mex.1", #Mexique
                "usa.1", #USA
                "arg.1", #Argentina
                "bre.1", #Brazil       
                "col.1", #Colombia
                "chi.1", #Chile
                "ecu.1", #Ecuador
                "par.1", #Paraguay         
                "jpn.1", #Japan
                "aus.1", #Australia              
              ]

list_teams = []
for competition in tqdm(Competitions):
    u = URL_prefix + competition
    get_page(u)
    soup = BeautifulSoup(driver.page_source, 'lxml')
    for games in soup.findAll(class_='Table__TR Table__TR--sm Table__even'):
        for linkGame in games.findAll(class_='hide-mobile'):
            for link in linkGame.findAll(class_='AnchorLink'):
                data_item = {"Competition": competition,
                             "LinkTeam": link["href"].replace("/club/", "/team/results/").replace(re.sub(".*/", "", link["href"].replace("/club/", "/team/results/")),"")
                            }
                list_teams.append(data_item)
    for games in soup.findAll(class_='filled Table__TR Table__TR--sm Table__even'):
        for linkGame in games.findAll(class_='hide-mobile'):
            for link in linkGame.findAll(class_='AnchorLink'):
                data_item = {"Competition": competition,
                             "LinkTeam": link["href"].replace("/club/", "/team/results/").replace(re.sub(".*/", "", link["href"].replace("/club/", "/team/results/")),"")
                            }
                list_teams.append(data_item)
Teams = pd.DataFrame(list_teams)
#Teams.head()

Hist = []
for i in tqdm(range(len(Teams))):
#for i in range(2):
    Year = []
    u = "https://www.espn.co.uk" + Teams.iloc[i]["LinkTeam"] + "league/" + Teams.iloc[i]["Competition"]
    get_page(u)
    soup = BeautifulSoup(driver.page_source, 'lxml')
    for j in soup.find_all(class_='Table__Title'):
        Year.append(j.text.split(",")[1])
    for r in soup.find_all(class_='headline headline__h1 dib'):
        team = r.text.replace(" Results","")
    i=0
    for cat in soup.find_all(class_='Table__ScrollerWrapper relative overflow-hidden'):
        for linkGame in cat.find_all(class_='Table__TR Table__TR--sm Table__even'):
            cols = linkGame.find_all(class_='Table__TD')
            for c in linkGame.find_all(class_='Table__TD'):
                if(c.find('span', class_='Table__Team score Custom_Notes_Header') is not None):
                    cols[2] = c.find('span', class_='Table__Team score Custom_Notes_Header')
            if datetime.datetime.strptime(cols[0].text + Year[i], "%a, %d %b %Y") > (datetime.datetime.strptime((datetime.datetime.now() + datetime.timedelta(days=1)).strftime("%Y-%m-%d"), "%Y-%m-%d")):
                Year[i] = str(int(Year[i]) - 1)
            data_item = {"Date": cols[0].text + Year[i], 
                         "Home": cols[1].text,
                         "Score": cols[2].text,
                         "Away": cols[3].text,
                         "Team": team,
                         "Competition": Teams.iloc[i]["Competition"]
                        }
            Hist.append(data_item)
        i=i+1

100%|██████████| 12/12 [00:19<00:00,  1.64s/it]


In [45]:
# import datetime
History = pd.DataFrame(Hist).drop_duplicates().drop("Team", axis=1)
History[['ScoreH', 'ScoreA']] = History['Score'].str.split(' - ', expand=True).astype(int)
History[['But']] = History['ScoreH'] + History['ScoreA']
History['Result'] = np.where(History['ScoreH'] == History['ScoreA'], 'Draw',
                             np.where(History['ScoreH'] > History['ScoreA'], 'Home', 'Away'))

History['Ds'] = History['Result'].apply(lambda x: 1 if x == 'Draw' else 0)
History['Hs'] = History['Result'].apply(lambda x: 1 if x == 'Home' else 0)
History['As'] = History['Result'].apply(lambda x: 1 if x == 'Away' else 0)
History['DA'] = History['Result'].apply(lambda x: 1 if x in ['Draw', 'Away'] else 0)
History['DH'] = History['Result'].apply(lambda x: 1 if x in ['Draw', 'Home'] else 0)
History['HA'] = History['Result'].apply(lambda x: 1 if x in ['Home', 'Away'] else 0)
History['LDEMs'] = np.where(History['ScoreH'] > 0, np.where(History['ScoreA'] > 0, 1, 0), 0)
History['LDEMPs'] = np.where(History['LDEMs'] == 1, 0, 1)

History['DALDEMs'] = np.where((History['LDEMs'] == 1) & (History['DA'] == 1), 1, 0)
History['DHLDEMs'] = np.where((History['LDEMs'] == 1) & (History['DH'] == 1), 1, 0)
History['HALDEMs'] = np.where((History['LDEMs'] == 1) & (History['HA'] == 1), 1, 0)
History['DALDEMPs'] = np.where((History['LDEMPs'] == 1) & (History['DA'] == 1), 1, 0)
History['DHLDEMPs'] = np.where((History['LDEMPs'] == 1) & (History['DH'] == 1), 1, 0)
History['HALDEMPs'] = np.where((History['LDEMPs'] == 1) & (History['HA'] == 1), 1, 0)

History["Date"] = History["Date"].apply(lambda x: datetime.datetime.strptime(x,  "%a, %d %b %Y").date())
History["IdGame"] = History["Date"].astype(str) + History["Home"] + History["Away"]
History['Date'] = pd.to_datetime(History['Date'], dayfirst=True)
History = History.sort_values('Date', ascending=False)

In [47]:
# Save History
if True:
    csv = pd.read_csv("History.csv")
    def bool(p):
        if any(csv['IdGame'].str.contains(str(p))):
            return(False)
        else:
            return(True)

    new = History.apply(lambda p : bool(p["IdGame"]), axis=1)
    csv = pd.concat([csv,History[new]])
    csv['Date'] = pd.to_datetime(csv['Date'])
    csv.sort_values(by='Date', ascending=False).drop_duplicates().to_csv('History.csv', index=False, encoding='utf-8')